# Create a graph for the possible future road network
For each component of stands that is not connected to any roads (respective: big roads), we wanna create a network of possible future road segments which strictly follow the boundaries of the forest stands.

## Imports & Settings

In [73]:
# Import standard libraries
import os
import re
import csv
import math
from collections import defaultdict, Counter

# Import third-party libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import missingno as msno

# Import geometrical and spatial libraries
from shapely.geometry import MultiPolygon, Polygon, Point, LineString
from shapely import wkt

# Import plotting libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib_scalebar.scalebar import ScaleBar


## Load data

In [74]:
# Set input and output paths
inpath = r"1_Preprocessed_Data\2_Stand_Components\unconnected_to_roads\2_nonsingle_inaccessible_components_with_ring_of_accessible_neighbors"
out_directory = r"1_Preprocessed_Data\3_Road_Network_Graphs"

In [75]:
# Initialize an empty dictionary
components = {}

# List all shapefiles in the directory
shapefiles = [f.path for f in os.scandir(inpath) if f.is_file() and f.name.endswith('.shp')]

for shapefile_path in shapefiles:
    # Extract the filename without extension
    filename = os.path.splitext(os.path.basename(shapefile_path))[0]
    
    # Extract the part after "combi_"
    key = filename.split('_stands')[0] if '_stands' in filename else filename
    
    # Load the shapefile into a GeoDataFrame
    components[key] = gpd.read_file(shapefile_path)
    
    print(f"Component '{key}' loaded with {len(components[key])} stands.")

Component 'component_10' loaded with 9 stands.
Component 'component_11' loaded with 3 stands.
Component 'component_12' loaded with 6 stands.
Component 'component_13' loaded with 7 stands.
Component 'component_14' loaded with 14 stands.
Component 'component_15' loaded with 5 stands.
Component 'component_16' loaded with 4 stands.
Component 'component_17' loaded with 6 stands.
Component 'component_18' loaded with 17 stands.
Component 'component_19' loaded with 4 stands.
Component 'component_1' loaded with 6 stands.
Component 'component_20' loaded with 5 stands.
Component 'component_21' loaded with 30 stands.
Component 'component_22' loaded with 10 stands.
Component 'component_23' loaded with 4 stands.
Component 'component_24' loaded with 12 stands.
Component 'component_25' loaded with 5 stands.
Component 'component_26' loaded with 10 stands.
Component 'component_2' loaded with 4 stands.
Component 'component_3' loaded with 91 stands.
Component 'component_4' loaded with 9 stands.
Component 

In [76]:
# load road data
roads = gpd.read_file(r'1_Preprocessed_Data\1_Roads_clean\roads_clean.shp')
roads = roads.to_crs(components['component_1'].crs)

## 1. Extract vertices and edges with attributes (slope, edge length) and exit points from boundaries [helper functions]
From the boundaries of the stands, we extract all points with coordianates and additionally save the edges (lines) connecting them. 

Note: We only want need exterior boundaries, and NO roads along interior boundaries of a forest stand (because such interior roads would not help with connecting the stands to the existing road network).

There will be helper functions for step I consider important enough to mention them seperately, even if the function might be a single line of code, it helps to beter understand the structure and what happens.

### 1.0 Snap coordinates to grid [helper function] 
Before comparing edges, snap all coordinates to a common grid (round coordinates to a fixed number of decimal places). This helps ensure that neighboring stands with slightly different coordinate representations share identical coordinates.

In [77]:
def snap_to_grid(coord, precision):
    """
    Snaps the given coordinates to the specified precision.

    Parameters:
        coord (tuple or list): A tuple or list containing the x, y (and optionally z) coordinates to be snapped.
        precision (int): The number of decimal places to round the coordinates.

    Returns:
        tuple: A tuple of coordinates rounded to the specified precision.

    Raises:
        ValueError: If the input is not a tuple or list, or if any coordinate is NaN or infinite.
    """
    # Check if input is a tuple or list
    if not isinstance(coord, (tuple, list)):
        raise ValueError("Input must be a tuple or list of coordinates")

    # Check for NaN or infinity values
    if any(map(lambda c: isinstance(c, (float, int)) and (math.isnan(c) or math.isinf(c)), coord)):
        raise ValueError("Coordinate contains NaN or infinite values")
    
    # Round coordinates to the specified precision
    return tuple(round(c, precision) for c in coord)

### 1.1 Calculate attributes [helper function]
To calculate the costs, we need to know for each edge its edge length and its approximative slope.
- *edgelength*: The length of an edge (u,v) is calculated using the Euclidean distance formula, to compute the distance between the coordinates of points u and v.
- *slope (approx.)*: The slope of an edge is approximated by the slope ("Declive") of the polygon the edge belongs to.


In [78]:
# Check geometry dimension for the one example component
component = next(iter(components.values()))  # Get the first component
geometry = component.geometry.iloc[0]

if geometry.has_z:
    print("The shapefile contains 3D geometries with Z-values (altitudes per coordinate).")
else:
    print("The shapefile contains 2D geometries (no altitude per coordinate).")
    
    # Check for altitude attribute
    if 'Altitude' in component.columns:
        print("The shapefile has an 'Altitude' column (single altitude per polygon).")
    else: 
        print("No 'Altitude' column found.")

The shapefile contains 2D geometries (no altitude per coordinate).
The shapefile has an 'Altitude' column (single altitude per polygon).


Note: If we knew the altitude per coordinate, the slope could be approximated via the ratio of the altitude difference to the edgelength; but the edges all belong to the same polygon so we only have one single altitude per polygon (which would lead to a slope of 0). Therefore, we just take the slope ("Declive") of the polygon.

In [79]:
def calculate_edge_length(u, v):
    """
    Calculate the length of the edge defined by two vertices (u, v).

    Parameters:
        u (tuple): A tuple representing the coordinates (x, y) of the first vertex.
        v (tuple): A tuple representing the coordinates (x, y) of the second vertex.

    Returns:
        float: The length of the edge between vertices u and v.

    Raises:
        ValueError: If the input coordinates are not tuples or lists of length 2.
    """
    # Ensure the input coordinates are valid (tuples of length 2)
    if not (isinstance(u, (tuple, list)) and len(u) == 2) or not (isinstance(v, (tuple, list)) and len(v) == 2):
        raise ValueError("Both u and v must be tuples or lists of length 2 representing coordinates.")

    # Create a LineString object to calculate the edge length
    line = LineString([u, v])

    # Return the length of the line
    return line.length

### 1.2 Extract nodes, edges, attributes [helper function]

In [80]:
def extract_boundaries_with_attributes(stands, precision=5):
    """
    Extracts the vertices, edges, edge attributes (such as edge length and slope),
    and exit points (intersections with roads) from polygon geometries in the input 
    stands dataset. Coordinates and intersections are snapped to a grid for precision.

    Parameters:
        stands (GeoDataFrame): A GeoDataFrame containing the stand boundaries and 
                                associated data. Each feature must have a polygon geometry.
        precision (int, optional, default=4): The precision to which the coordinates 
                                              will be snapped when extracted (controls decimal places).

    Returns:
        tuple: A tuple containing four elements:
            - vertices (list of tuples): A list of unique vertices (coordinates of stand boundary points)
            - edges (list of tuples): A list of edges (pairs of vertices)
            - edge_attributes (list of dicts): A list of dictionaries containing edge attributes (edge length, slope)
            - exit_points (list of tuples): A list of exit points (coordinates where stand boundaries intersect roads)
    """
    vertices = set()  # Using set to avoid duplicate vertices
    edges = []
    edge_attributes = []
    exit_points = []

    for _, feature in stands.iterrows():
        geometry = feature.geometry
        slope = feature['Declive']

        # Process only polygons
        if geometry.geom_type == 'Polygon':
            # Snap coordinates for the exterior
            exterior_coords = [snap_to_grid(coord, precision) for coord in geometry.exterior.coords]
            vertices.update(exterior_coords)  # Use set to avoid duplicates

            # Creating edges
            for i in range(len(exterior_coords) - 1):
                u, v = exterior_coords[i], exterior_coords[i + 1]
                edges.append((u, v))

                # Edge length and slope
                edge_length = calculate_edge_length(u, v)
                edge_attributes.append({'edgelength': edge_length, 'slope': slope, 'has_exit': False})

            # Find intersections with roads (exit points)
            stand_boundary = geometry.exterior  # Stand polygon boundary
            for road in roads.geometry:
                if stand_boundary.intersects(road):  # Only proceed if there is an intersection
                    intersection = stand_boundary.intersection(road)
                    
                    # Process intersections
                    if intersection.geom_type == 'Point':
                        exit_points.append(snap_to_grid((intersection.x, intersection.y), precision))
                    elif intersection.geom_type == 'MultiPoint':
                        exit_points.extend([snap_to_grid((point.x, point.y), precision) for point in intersection.geoms])

    return list(vertices), edges, edge_attributes, exit_points

## 2. Create the Graph
Next, we'll use these vertices and edges to build a graph. This graph will represent the potential road network, where the roads are constrained to follow the boundaries of the forest stands.

### Storing graph data [helper function] 

In [81]:
def store_data_for_graph(out_path, vertices, edges, attributes, name):
    """
    Saves the data for the graph (vertices, edges, and attributes) into CSV files within a specified folder.

    Parameters:
        out_path (str): The directory where the CSV files will be saved.
        vertices (list): A list of vertex coordinates (x, y) to be saved.
        edges (list): A list of edges, each represented by a tuple of vertices.
        attributes (list): A list of dictionaries containing edge attributes (e.g., 'edgelength', 'slope').
        name (str): The name of the component, used in the file naming.
        G (networkx.Graph): The graph containing node attributes, including `is_exit`.
    """

    # Ensure the folder exists
    os.makedirs(out_path, exist_ok=True)

    # File paths
    vertices_file = os.path.join(out_path, f'nodes_{name}.csv')
    edges_file = os.path.join(out_path, f'edges_{name}.csv')
    attributes_file = os.path.join(out_path, f'attributes_{name}.csv')
    edges_with_attributes_file = os.path.join(out_path, f'edges_with_attributes_{name}.csv')

    # Save vertices to CSV with the 'is_exit' column
    vertices_data = [{'x': x, 'y': y, } for x, y in vertices]
    pd.DataFrame(vertices_data).to_csv(vertices_file, index=False)

    # Save edges to CSV
    edges_data = [(f"({u[0]}, {u[1]})", f"({v[0]}, {v[1]})") for u, v in edges]
    pd.DataFrame(edges_data, columns=['Node1(x,y)', 'Node2(x,y)']).to_csv(edges_file, index=False)

    # Save attributes to CSV
    pd.DataFrame(attributes).to_csv(attributes_file, index=False)

    # Save edges with attributes to CSV
    edges_attributes_data = [
        {
            'Node1(x,y)': f"({u[0]}, {u[1]})",
            'Node2(x,y)': f"({v[0]}, {v[1]})",
            'edgelength': attributes['edgelength'],
            'slope': attributes['slope'],
            'has_exit': attributes['has_exit']
        }
        for (u, v), attributes in zip(edges, attributes)
    ]
    pd.DataFrame(edges_attributes_data).to_csv(edges_with_attributes_file, index=False)


### Create graph from vertices, edges, attributes [helper function] 

In [82]:
def create_graph(vertices, edges, attributes):
    """
    Create an undirected graph using NetworkX based on the provided vertices, edges, and edge attributes.

    Parameters:
        vertices (list): A list of vertices (nodes) to be added to the graph.
        edges (list): A list of edges represented as tuples (u, v), where u and v are vertices.
        attributes (list): A list of dictionaries, where each dictionary contains edge attributes 
                           (such as 'weight', 'length', 'has_exit', etc.) corresponding to each edge.

    Returns:
        nx.Graph: A NetworkX Graph object containing the nodes, edges, and associated attributes.
    
    Example:
        vertices = [(x1, y1), (x2, y2), (x3, y3)]
        edges = [((x1, y1), (x2, y2)),
                 ((x2, y2), (x3, y3))]
        attributes = [
            {'edgelength': 74.92801170857655, 'slope': 18.0777, 'has_exit': False},
            {'edgelength': 61.5825216583405, 'slope': 18.0777, 'has_exit': False}
        ]
    """

    if len(edges) != len(attributes):
        raise ValueError("The number of edges and attributes must match.")

    G = nx.Graph()
    for vertex in vertices:
        G.add_node(vertex, is_exit=False)  # Default: not an exit

    for edge, attr in zip(edges, attributes):
        u, v = edge
        # Add the edge with the attributes
        G.add_edge(u, v, **attr)

    return G

### Plot graph and store image [helper function]

In [83]:
def plot_and_save(outpath, G, name, prefix):
    """Plots the graph 'G' along with associated geographical components and saves the plot to a file.

    Parameters:
        G (nx.Graph): The NetworkX graph object to be plotted.
        name (str): The name of the component, used in the plot title and file naming.
        filename (str): The path and filename where the plot will be saved.

    Saves:
        A PNG image containing:
            - The geographical component from the 'components' GeoDataFrame.
            - The graph 'G' overlaid with nodes and edges.
            - A summary of node degrees, total edge length, and number of exit nodes as an annotation.
    """
    fig = plt.figure(figsize=(10, 10))
    
    filename=os.path.join(outpath,f'{prefix}_{name}.png')

    components[name].plot(ax=plt.gca(), color='lightgray', alpha=0.5)  # Add GeoDataFrame
    components[name].apply(lambda row: plt.annotate(text=row['ID_UG'], 
                                                xy=(row.geometry.centroid.x, row.geometry.centroid.y), 
                                                xytext=(3, 3), textcoords="offset points", 
                                                fontsize=8, color='black'), axis=1)

    
    nodecolor = ["blue" if G.nodes[node].get('is_exit', False) else "red" for node in G.nodes()]

    nx.draw(G, pos={node: node for node in G.nodes()}, node_size=50, 
            node_color=nodecolor, edge_color="blue", with_labels=False)
    
    # Get degree of each node
    degree_counts = Counter(dict(G.degree()).values())

    # Calculate total edge length
    total_edge_length = sum(G[u][v].get('edgelength', 0) for u, v in G.edges())

    # Calculate total costs across multiple attributes
    # List of required attributes
    cost_attributeskeys = ['Build5m', 'Maintain5m', 'Build10m', 'Maintain10m', 'Upgrade']

    # Check if all required attributes exist in any edge
    costattributes_exist = all(any(attr in G[u][v] for u, v in G.edges()) for attr in cost_attributeskeys)

    if costattributes_exist:

        # Calculate total costs if the attributes exist
        total_costs = sum(
            G[u][v].get('Build5', 0) + 
            G[u][v].get('Maintain5', 0) + 
            G[u][v].get('Build10', 0) + 
            G[u][v].get('Maintain10', 0) + 
            G[u][v].get('Upgrade', 0)
            for u, v in G.edges()
        )

    # Count nodes marked as exit
    exit_nodes_count = sum(1 for node in G.nodes if G.nodes[node].get('is_exit'))

    # Create the overview text
    degree_overview = "Graph Degree Overview:\n"
    for degree, count in sorted(degree_counts.items()):
        degree_overview += f"Nodes with degree {degree}: {count}\n"

    degree_overview += f"\nNumber of Exit Nodes: {exit_nodes_count}\n"
    degree_overview += f"\nTotal Edge Length: {total_edge_length:.2f}\n"
    if costattributes_exist:
        degree_overview += f"Total Costs: {total_costs:.2f}\n"

    # Add graph details to plot
    plt.title(f"Graph Plot for {name},\n {len(G.nodes)} nodes\n {len(G.edges)} edges")
    
    # Add the degree overview text to the plot
    plt.text(0.5, -0.1, degree_overview, ha='center', va='top', transform=plt.gca().transAxes, fontsize=10)

    plt.savefig(filename, dpi=300, bbox_inches='tight')  # Save plot
    plt.close(fig)


## 3. Handle exit points [helper functions]

### find nearest node [helper function]

In [84]:
def find_nearest_node(graph, exit_points):
    """
    Find the nearest nodes in the graph for a list of exit points based on Euclidean distance.

    Parameters:
        graph (nx.Graph): A NetworkX graph object containing nodes and their coordinates.
        exit_points (list): A list of exit points, where each exit point is a tuple representing (x, y) coordinates.

    Returns:
        pd.DataFrame: A DataFrame with columns ['exit_point', 'nearest_neighbor_node', 'distance'],
                      containing the exit points, their corresponding nearest nodes, and the calculated distances.

    Example:
        graph = nx.Graph()
        exit_points = [(x1, y1), (x2, y2)]
        result_df = find_nearest_node(graph, exit_points)
    """
    results = []
    for exit_point in exit_points:
        # Find the nearest node based on distance
        nearest_node = min(graph.nodes, key=lambda node: np.linalg.norm(np.array(exit_point) - np.array(node)))
        
        # Calculate the distance to the nearest node
        distance = np.linalg.norm(np.array(exit_point) - np.array(nearest_node))
        
        # Append the result
        results.append((exit_point, nearest_node, distance))
    
    # Create DataFrame
    df = pd.DataFrame(results, columns=['exit_point', 'nearest_neighbor_node', 'distance'])
    return df

### find edges closeby [helper function]

In [85]:
def find_closest_edge(G, exit_x, exit_y):
    """
    Finds the closest edge in the graph to the exit point (exit_x, exit_y).
    Returns the edge (u, v) and the intersection point.
    """
    min_dist = float('inf')
    closest_edge = None
    intersection_point = None

    # Iterate through all edges in the graph
    for u, v in G.edges():
        # Get the coordinates of the nodes (assumed to be stored as node attributes)
        ux, uy = u
        vx, vy = v
        
        # Create a LineString for the edge
        line = LineString([(ux, uy), (vx, vy)])

        # Create a Point for the exit
        exit_point = Point(exit_x, exit_y)

        # Check if the exit point intersects the edge
        if line.distance(exit_point) < min_dist:
            min_dist = line.distance(exit_point)
            closest_edge = (u, v)
            # If the point is on the line, we can get the intersection point
            if line.intersects(exit_point):
                intersection_point = line.interpolate(line.project(exit_point))  # Exact intersection point

    return closest_edge, intersection_point

### split at exit [helper function]

In [86]:
def split_edge_at_exit(G, exit_node, exit_x, exit_y):
    """
    Splits the edge in the graph that is closest to the exit node based on coordinates.
    After splitting, ensures that the new edges have the 'has_exit' attribute if they contain the exit node.
    Also adds the 'has_exit' attribute to the edge if the exit node is already part of it.

    Parameters:
        G (nx.Graph): The NetworkX graph containing the edges to be split.
        exit_node (tuple): The coordinates (x, y) of the exit node to be added.
        exit_x (float): The x-coordinate of the exit point.
        exit_y (float): The y-coordinate of the exit point.

    Actions:
        - Identifies the closest edge to the exit point.
        - Splits the identified edge by adding the exit node and connecting it to both vertices of the edge.
        - Removes the original edge and adds two new edges that include the exit node.
        - Adds the 'has_exit' attribute to the edges, whether split or not.
    """
    # Find the closest edge and the intersection point
    closest_edge, intersection_point = find_closest_edge(G, exit_x, exit_y)

    if closest_edge is None:
        print(f"Warning: No edge found for exit point {exit_node}. Skipping split.")
        return

    u, v = closest_edge

    # If the exit_node is already part of the edge, just mark the edge with 'has_exit'
    if exit_node == u or exit_node == v:
        print(f"Exit node {exit_node} is already part of the edge ({u}, {v}). Marking it with 'has_exit'.")
        G[u][v]['has_exit'] = True
        return
    
    # Get attributes from the original edge
    original_slope = G[u][v].get('slope', None)

    # Remove the original edge
    G.remove_edge(u, v)

    # Add the exit node and split the edge
    G.add_edge(u, exit_node)
    G.add_edge(exit_node, v)

    # Assign attributes to the new edges
    for node1, node2 in [(u, exit_node), (exit_node, v)]:
        G[node1][node2]['has_exit'] = True
        G[node1][node2]['slope'] = original_slope  # Retain the slope
        G[node1][node2]['edgelength'] = calculate_edge_length(node1, node2)  # Recalculate edge length

### handling exit points [helper function] [mini workflow]

In [87]:
def handle_exit_points(G, exit_points):
    """
    Processes exit points by adding them as nodes or merging them with existing ones.
    Also updates edge attributes to indicate if they contain an exit node.

    Returns:
        tuple: (count of already contained exit points, count of newly added exit points).
    """
    # Find nearest nodes and determine new exit points
    exitdf = find_nearest_node(G, exit_points)
    exitdf['newexit'] = np.where(exitdf['distance'] <= 10, exitdf['nearest_neighbor_node'], exitdf['exit_point'])

    # Add exit nodes to graph
    for node in exitdf['newexit']:
        G.add_node(node, is_exit=True)  # Mark node as an exit

    # Split edges at new exit points if necessary
    for _, row in exitdf[exitdf['distance'] > 10].iterrows():
        exit_x, exit_y = row['exit_point']
        closest_edge, intersection_point = find_closest_edge(G, exit_x, exit_y)

        if closest_edge:
            split_edge_at_exit(G, row['newexit'], exit_x, exit_y)

    # Mark edges that contain exit nodes
    exit_nodes = set(exitdf['newexit'])  # Convert to set for fast lookup
    for u, v in G.edges():
        G[u][v]['has_exit'] = u in exit_nodes or v in exit_nodes  

    # Return counts
    return (exitdf['distance'] <= 10).sum(), (exitdf['distance'] > 10).sum()

### TO DO: Verify Exit Points (Plot with roads)

### store updated graph data [helper function]

In [88]:
def store_updated_graph_data(out_path, G, name, prefix):
    """
    Saves the updated graph data (vertices, edges, and attributes) into CSV files after processing exit points.

    Parameters:
        out_path (str): The directory where the CSV files will be saved.
        name (str): The name of the component, used in the file naming.
    """
    # Ensure the folder exists
    os.makedirs(out_path, exist_ok=True)

    # File paths
    updated_vertices_file = os.path.join(out_path, f'{prefix}_nodes_{name}.csv')
    updated_edges_file = os.path.join(out_path, f'{prefix}_edges_{name}.csv')
    updated_attributes_file = os.path.join(out_path, f'{prefix}_attributes_{name}.csv')
    updated_edges_with_attributes_file = os.path.join(out_path, f'{prefix}_edges_with_attributes_{name}.csv')

    # Extract and save vertices (including 'is_exit' flag if present)
    vertices_data = [{'x': node[0], 'y': node[1], 'is_exit': G.nodes[node].get('is_exit')} for node in G.nodes]
    pd.DataFrame(vertices_data).to_csv(updated_vertices_file, index=False)

    # Extract and save edges
    edges_data = [(f"({u[0]}, {u[1]})", f"({v[0]}, {v[1]})") for u, v in G.edges]
    pd.DataFrame(edges_data, columns=['Node1(x,y)', 'Node2(x,y)']).to_csv(updated_edges_file, index=False)

    # Extract and save attributes
    attributes_data = [{**G[u][v]} for u, v in G.edges]
    pd.DataFrame(attributes_data).to_csv(updated_attributes_file, index=False)

    # Extract and save edges with attributes dynamically
    edges_attributes_data = []
    for u, v in G.edges:
        edge_data = {'Node1(x,y)': f"({u[0]}, {u[1]})", 'Node2(x,y)': f"({v[0]}, {v[1]})"}
        edge_data.update(G[u][v])  # Dynamically add all attributes
        edges_attributes_data.append(edge_data)
    pd.DataFrame(edges_attributes_data).to_csv(updated_edges_with_attributes_file, index=False)


## 4. Calculate the costs associated with each road segment

There are costs associated with the construction, maintenance and upgrade of roads, depending on the slope. They need to be calculated for each road segment according to their length.

We name the 3 categories of slope:
* flat (slope ≤ 5)
* moderate (5 < slope < 25)
* steep (slope ≥ 25)

### Load costs

In [89]:
costs = {
    "flat": {  
        "Build5m": 2147,
        "Maintain5m": 1073.5
    },
    "moderate": {
        "Build5m": 0,
        "Maintain5m": 0
    },
    "steep": {
        "Build5m": 7514.5,
        "Maintain5m": 2683.75
    }
}
costs_df = pd.DataFrame(costs).T
print(costs_df)

          Build5m  Maintain5m
flat       2147.0     1073.50
moderate      0.0        0.00
steep      7514.5     2683.75


### approximation for unknown costs

In [90]:
costs_df['Build10m'] = 1.5 * costs_df.Build5m
costs_df['Maintain10m'] = costs_df.Maintain5m
costs_df['Upgrade'] = 0.75 * costs_df.Build5m
costs_df

,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade
flat,2147.0,1073.50,3220.50,1073.50,1610.250
moderate,0.0,0.00,0.00,0.00,0.000
steep,7514.5,2683.75,11271.75,2683.75,5635.875


In [91]:
costs_df.iloc[1] = (costs_df.iloc[0] + costs_df.iloc[2]) / 2
costs_df

,Build5m,Maintain5m,Build10m,Maintain10m,Upgrade
flat,2147.00,1073.500,3220.500,1073.500,1610.2500
moderate,4830.75,1878.625,7246.125,1878.625,3623.0625
steep,7514.50,2683.750,11271.750,2683.750,5635.8750


### assign the costs [helper function]

In [92]:
def assign_all_costs_to_edges(G):
    """
    Assign all cost-related variables (Build5m, Maintain5m, Upgrade, Build10m, Maintain10m) to edges
    based on slope, road type, and edge length.
    
    Parameters:
    G (NetworkX graph): The graph representing the road network.
    
    Returns:
    G (NetworkX graph): The graph with updated cost attributes.
    """

    # Iterate over each edge in the graph and set the new attributes
    for u, v, data in G.edges(data=True):
        if 'edgelength' in data and 'slope' in data:  # Ensure both 'edgelength' and 'slope' exist

            # Determine the correct category based on slope
            if data['slope'] <= 5:
                category = "flat"
            elif 5 < data['slope'] < 25:
                category = "moderate"
            else:  # data['slope'] >= 25
                category = "steep"

            # Assign costs based on the selected category
            for column in costs_df.columns:
                data[column] = data['edgelength'] * costs_df.loc[category, column]
                
    print(f'{name} costs assigned')
    return G


## 5. Merge short edges to improve performance of optimisation algorithms

Note: It would have been possible to merge earlier, but the cost approximization is better this way, because we have more information on the slope if we don't merge the edges before calculation. 

### merge short edges [helper function] 

In [93]:
def merge_short_edges(G, length_threshold):
    """
    Merges edges in a graph G that are shorter than a given length_threshold and connect nodes with degree 2.
    The function merges two such edges by summing their lengths and other attributes. If at least one of the original 
    edges has an exit, the merged edge will also have an exit. The function removes nodes involved in the merge if 
    they are not exit nodes.

    Parameters:
    G (networkx.Graph): The input graph containing nodes and edges to be merged.
    length_threshold (float): The maximum length below which edges will be merged.

    Returns:
    networkx.Graph: The modified graph with merged edges.
    """
    # While there are edges to merge (based on the length threshold)
    while any(
        data.get('edgelength', float('inf')) < length_threshold and 
        G.degree[v] == 2
        for u, v, data in G.edges(data=True)
    ):
        # Track if any merge occurs in this iteration
        merged = False

        # Iterate over a static list of edges
        for u, v, data_uv in list(G.edges(data=True)):
            # Skip merging if node v is an exit node or a crossing point
            if G.nodes[v].get('is_exit') or G.degree[v] != 2:
                continue  # Don't attempt to merge if node v is an exit node or a crossing

            if data_uv.get('edgelength', float('inf')) < length_threshold:
                neighbors = list(G.neighbors(v))

                # Only proceed if the node v has exactly 2 neighbors as it will be the one removed
                if G.degree[v] == 2:#true
                    # Initialize variables to find the shortest neighbor
                    shortest_length = float('inf')
                    shortest_neighbor = None

                    # Find the shortest neighbor n (of node v) to merge
                    for n in neighbors:
                        if n != u and G.degree[n] == 2:
                            edge_length = G[v][n].get('edgelength', float('inf'))
                            if edge_length < shortest_length:
                                shortest_length = edge_length
                                shortest_neighbor = n

                    # If a shortest neighbor is found, perform the merge
                    if shortest_neighbor is not None:
                        # Edge data for the second edge
                        data_vn = G[v][shortest_neighbor]

                        # Compute merged length
                        new_length = data_uv['edgelength'] + data_vn['edgelength']

                        # Check if either edge has an exit
                        has_exit = data_uv.get('has_exit') or data_vn.get('has_exit')

                        # Create new edge data by merging attributes
                        new_data = {
                            'edgelength': new_length,
                            'has_exit': has_exit,  # Set 'has_exit' based on the original edges
                            'Build5m': data_uv.get('Build5m', 0) + data_vn.get('Build5m', 0),
                            'Maintain5m': data_uv.get('Maintain5m', 0) + data_vn.get('Maintain5m', 0),
                            'Build10m': data_uv.get('Build10m', 0) + data_vn.get('Build10m', 0),
                            'Maintain10m': data_uv.get('Maintain10m', 0) + data_vn.get('Maintain10m', 0),
                            'Upgrade': data_uv.get('Upgrade', 0) + data_vn.get('Upgrade', 0),
                        }

                        # Add merged edge
                        G.add_edge(u, shortest_neighbor, **new_data)

                        # Remove old edges
                        if G.has_edge(u, v):
                            G.remove_edge(u, v)
                        if G.has_edge(v, shortest_neighbor):
                            G.remove_edge(v, shortest_neighbor)

                        # Remove node v
                        if G.has_node(v):
                            G.remove_node(v)

                        # Mark as merged and break out to restart
                        merged = True
                        break

            if merged:
                break

        # If no merges were performed, we're done
        if not merged:
            break

    return G

### verify the result of merging [helper function]

In [94]:
def verify_merge(G_before_merge, G_after_merge):
    """
    Verifies the merge by comparing the total sum of `edgelength` and costs 
    before and after the merge.

    Parameters:
        G_before_merge: The graph before merging.
        G_after_merge: The graph after merging.
    """
    def calculate_totals(G):
        return {
            'edgelength': sum(data.get('edgelength', 0) for _, _, data in G.edges(data=True)),
            'Build5m': sum(data.get('Build5m', 0) for _, _, data in G.edges(data=True)),
            'Maintain5m': sum(data.get('Maintain5m', 0) for _, _, data in G.edges(data=True)),
            'Build10m': sum(data.get('Build10m', 0) for _, _, data in G.edges(data=True)),
            'Maintain10m': sum(data.get('Maintain10m', 0) for _, _, data in G.edges(data=True)),
            'Upgrade': sum(data.get('Upgrade', 0) for _, _, data in G.edges(data=True))
        }
    
    totals_before = calculate_totals(G_before_merge)
    totals_after = calculate_totals(G_after_merge)
    
    # Check for differences
    differences = {key: round(totals_after[key] - totals_before[key], 2) for key in totals_before}
    significant_differences = {k: v for k, v in differences.items() if v != 0}
    
    if significant_differences:
        print("Warning: Differences detected in the following totals:")
        for key, value in significant_differences.items():
            print(f"  {key}: {value}")
    
    return {
        'deleted_edges': G_before_merge.number_of_edges() - G_after_merge.number_of_edges()
    }


## Store debugging infos [helper function]

In [95]:
def write_debug_infos(base_info, folder_path, name):
    # Debugging Infos
        total_vertices = len(vertices)
        total_edges = len(edges)
        #unique_road_segments = len(road_segments)
        #merged_edges = total_edges - unique_road_segments

        # Write info for the component to its individual text file
        info_file_path = os.path.join(folder_path, f'info_{name}.txt')
        with open(info_file_path, 'w') as info_file:
            info_file.write(f"Component {name} Information\n")
            info_file.write(f"----------------------------\n")
            info_file.write(f"Number of vertices: {total_vertices}\n")
            info_file.write(f"Total original edges: {total_edges}\n")
            #info_file.write(f"Unique road segments after removing duplicate (shared) edges: {unique_road_segments}\n")
            #info_file.write(f"Number of removed edges: {merged_edges}\n")
            info_file.write(f"Number of edge attributes: {len(attributes)}\n")
            info_file.write(f"Number of deleted edges due to merging of short ones: {results['deleted_edges']}\n")
            info_file.write(f"Totals Before Merge: {results['before']}\n")
            info_file.write(f"Totals After Merge: {results['after']}\n")
            info_file.write(f"Differences: {results['difference']}\n")
            info_file.write(f"Number of nodes per degree: {results['node_degrees']}\n")
            info_file.write(f"----------------------------\n")

        # Write the aggregated summary to the base info file
        base_info.write(f"\nComponent {name} Information\n")
        base_info.write(f"----------------------------\n")
        base_info.write(f"Number of vertices: {total_vertices}\n")
        base_info.write(f"Total original edges: {total_edges}\n")
        #base_info.write(f"Unique road segments after removing duplicate (shared) edges: {unique_road_segments}\n")
        #base_info.write(f"Number of removed edges: {merged_edges}\n")
        base_info.write(f"Number of edge attributes: {len(attributes)}\n")
        base_info.write(f"Number of deleted edges due to merging of short ones: {results['deleted_edges']}\n")
        base_info.write(f"Totals Before Merge: {results['before']}\n")
        base_info.write(f"Totals After Merge: {results['after']}\n")
        base_info.write(f"Differences: {results['difference']}\n")
        base_info.write(f"Number of nodes per degree: {results['node_degrees']}\n")
        base_info.write(f"----------------------------\n")


## 6. Handle source nodes

### create source nodes for each stand [helper function]

### assign costs to source nodes [helper function]

## Workflow Step 1-5

In [ ]:
### WORKFLOW ###
# Define output path for aggregated component information
#base_info_file = f'{out_directory}/info.txt'
#write_base_info_header(base_info_file)  # Write header for summary file

for name, df in components.items():
    print(f"Processing component: {name}")

    # Create a dedicated folder for the current component
    component_folder = f"{out_directory}/{name}"
    os.makedirs(component_folder, exist_ok=True)

    # Extract key graph-related data from the component's dataframe
    vertices, edges, attributes, exit_points = extract_boundaries_with_attributes(df)

    # Save the extracted data for further use
    store_data_for_graph(component_folder, vertices, edges, attributes, name)

    # Construct the initial graph representation
    G = create_graph(vertices, edges, attributes)

    # Visualize and save the initial graph
    plot_and_save(component_folder, G, name, prefix='0graph')

    # Process exit points: determine which are contained and which need to be added
    contained_count, to_add_count = handle_exit_points(G, exit_points)
    print(f"{contained_count} exit points already exist")
    print(f"{to_add_count} exit points added")

    # Update and save the graph after handling exit points
    plot_and_save(component_folder, G, name, prefix='1withexits_graph')

    # Store the updated graph data, including nodes and edges with attributes
    store_updated_graph_data(component_folder, G, name, prefix='1withexits')   

    #### ASSIGNING COSTS TO EDGES ####

    # Assign cost values to all edges in the graph
    G = assign_all_costs_to_edges(G)

    # Save and visualize the graph with assigned edge costs
    store_updated_graph_data(component_folder, G, name, prefix='2withcosts')
    plot_and_save(component_folder, G, name, prefix='2withcosts_graph')

    #### MERGING SHORT EDGES ####

    # Create a copy of the graph before merging short edges for comparison
    G_before_merge = G.copy()
        
    # Merge edges that are shorter than the specified length threshold
    G = merge_short_edges(G, length_threshold=1000)

    # Visualize and save the updated graph after merging short edges
    plot_and_save(component_folder, G, name, prefix='3aftermerge_graph')

    # Store the updated graph data with merged edges
    store_updated_graph_data(component_folder, G, name, prefix='3aftermerge') 

    # Uncomment if you want to write debug information for further analysis
    # write_debug_infos(base_info, component_folder, name)

# Once all components are processed, a summary file containing aggregated information is saved
# print(f"Aggregated information for all components saved in {base_info_file}")


Processing component: component_10
2 exit points already exist
2 exit points added
component_10 costs assigned
Processing component: component_11
1 exit points already exist
1 exit points added
component_11 costs assigned
Processing component: component_12
7 exit points already exist
8 exit points added
component_12 costs assigned
Processing component: component_13
Exit node (-14725.76646, 148708.12995) is already part of the edge ((-14854.02958, 148755.56971), (-14725.76646, 148708.12995)). Marking it with 'has_exit'.
Exit node (-14511.22747, 148637.70934) is already part of the edge ((-14568.27898, 148670.90301), (-14511.22747, 148637.70934)). Marking it with 'has_exit'.
Exit node (-14482.92326, 148621.2414) is already part of the edge ((-14451.86208, 148603.16941), (-14482.92326, 148621.2414)). Marking it with 'has_exit'.
Exit node (-14433.07574, 148589.79609) is already part of the edge ((-14451.86208, 148603.16941), (-14433.07574, 148589.79609)). Marking it with 'has_exit'.
8 exit